# Clase 2: Seguimiento y comparación de modelos con MLflow

Dataset: Churn de clientes - Telco

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

url = "C:/MLOPS/clase_mlops/Proyecto_1/data/WA_Fn-UseC_-Telco-Customer-Churn.csv"
df = pd.read_csv(url)
df.drop(['customerID'], axis=1, inplace=True)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.dropna(inplace=True)
df = pd.get_dummies(df, drop_first=True)

X = df.drop('Churn_Yes', axis=1)
y = df['Churn_Yes']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [2]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score

mlflow.set_experiment("Churn_Telco_Comparacion")

2025/04/10 19:16:51 INFO mlflow.tracking.fluent: Experiment with name 'Churn_Telco_Comparacion' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/MLOPS/clase_mlops/Proyecto_1/notebooks/mlruns/547667526282479684', creation_time=1744330611488, experiment_id='547667526282479684', last_update_time=1744330611488, lifecycle_stage='active', name='Churn_Telco_Comparacion', tags={}>

### Entrenamiento: Logistic Regression

In [3]:
with mlflow.start_run(run_name="Logistic Regression"):
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]

    mlflow.log_param("modelo", "logistic")
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("auc", roc_auc_score(y_test, y_proba))
    
    mlflow.sklearn.log_model(model, "modelo_logistico")

c:\MLOPS\clase_mlops\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
2025/04/10 19:21:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Entrenamiento: Random Forest

In [4]:
with mlflow.start_run(run_name="Random Forest"):
    model = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]

    mlflow.log_param("modelo", "random_forest")
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 6)
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("auc", roc_auc_score(y_test, y_proba))
    
    mlflow.sklearn.log_model(model, "modelo_rf")

2025/04/10 19:21:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Support Vector Machine (SVM) 

In [ ]:
from sklearn.svm import SVC

with mlflow.start_run(run_name="SVM"):
    # Utilizando Support Vector Machine
    model = SVC(kernel='linear', C=1, probability=True, random_state=42)  # Ajuste del parámetro 'kernel' y 'C'
    model.fit(X_train, y_train)
    
    # Predicciones
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:,1]

    # Log de los parámetros y métricas en MLflow
    mlflow.log_param("modelo", "svm")
    mlflow.log_param("kernel", "linear")
    mlflow.log_param("C", 1)
    
    mlflow.log_metric("accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("f1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("recall", recall_score(y_test, y_pred))
    mlflow.log_metric("auc", roc_auc_score(y_test, y_proba))
    
    # Guardar el modelo SVM en MLflow
    mlflow.sklearn.log_model(model, "modelo_svm")


### Visualizar Experimentos en la UI de MLflow

In [ ]:
# Ejecuta en terminal (no dentro del notebook):
# mlflow ui
# Luego abre en el navegador: http://127.0.0.1:5000